In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.random as nr
import sklearn.model_selection as ms
import seaborn as sns
from sklearn.model_selection import cross_validate
import sklearn.metrics as sklm
from sklearn import linear_model
from sklearn import preprocessing
import math
import scipy.stats as ss
from sklearn import feature_selection as fs
from xgboost import XGBClassifier
import xgboost as xgb

%matplotlib inline

In [2]:
features = pd.read_csv("train.csv")

In [3]:
test = pd.read_csv('test.csv')
test.shape

(16496, 18)

In [4]:
def new_name(data, cols):
    for col in cols:
        data[col + '_Str'] = data[col].astype(str)
    
cols = ['Year_of_birth', 'Year_of_recruitment']
new_name(features, cols)
new_name(test, cols)

In [5]:
features = features.drop(['Year_of_recruitment', 'Year_of_birth'], axis = 1)
test = test.drop(['Year_of_recruitment', 'Year_of_birth'], axis = 1)

print(features.shape)
print(test.shape)

(38312, 19)
(16496, 18)


In [6]:
def value_count(data, cols):
    for col in cols:
        value = data[col].value_counts().count()
        print('\n' + 'For column' + '                                     ' + col)
        print(value)
        
colss = ['Division', 'Qualification', 'Gender', 'Channel_of_Recruitment',
       'Trainings_Attended', 'Year_of_birth_Str', 'Last_performance_score', 'Targets_met', 'Previous_Award',
       'Training_score_average', 'State_Of_Origin', 'Foreign_schooled',
       'Marital_Status', 'Past_Disciplinary_Action',
       'Previous_IntraDepartmental_Movement', 'No_of_previous_employers',
       'Promoted_or_Not', 'Year_of_birth_Str']


value_count(features, colss)


For column                                     Division
9

For column                                     Qualification
3

For column                                     Gender
2

For column                                     Channel_of_Recruitment
3

For column                                     Trainings_Attended
10

For column                                     Year_of_birth_Str
49

For column                                     Last_performance_score
6

For column                                     Targets_met
2

For column                                     Previous_Award
2

For column                                     Training_score_average
61

For column                                     State_Of_Origin
37

For column                                     Foreign_schooled
2

For column                                     Marital_Status
3

For column                                     Past_Disciplinary_Action
2

For column                                     Previous_Int

In [7]:
year_categories = {'1991': '1990-2001', '1990': '1990-2001', '1989': '1980-1989', '1992': '1990-2001',
                   '1988': '1980-1989', '1987': '1980-1989', '1993': '1990-2001', '1994': '1990-2001',
                  '1986': '1980-1989', '1985': '1980-1989', '1984': '1980-1989', '1995': '1990-2001', 
                  '1983': '1980-1989', '1982': '1980-1989', '1981': '1980-1989', '1980': '1980-1989',
                  '1996': '1990-2001', '1979': '1970-1979', '1978': '1970-1979', '1977': '1970-1979',
                   '1976': '1970-1979', '1975': '1970-1979', '1997': '1990-2001', '1973': '1970-1979',
                   '1974': '1970-1979', '1971': '1970-1979', '1972': '1970-1979', '1970': '1970-1979',
                   '1969': '1950-1969', '1968': '1950-1969', '1998': '1990-2001', '1966': '1950-1969',
                   '1967': '1950-1969', '1965': '1950-1969', '1964': '1950-1969', '1963': '1950-1969',
                   '1961': '1950-1969', '1962': '1950-1969', '1999': '1990-2001', '2001': '1990-2001',
                   '2000': '1990-2001', '1957': '1950-1969', '1956': '1950-1969', '1955': '1950-1969',
                   '1950': '1950-1969', '1952':'1950-1969', '1958': '1950-1969', '1959': '1950-1969',
                   '1960': '1950-1969'}

features['Year_of_birth_Str'] = [year_categories[x] for x in features['Year_of_birth_Str']]
print(features['Year_of_birth_Str'].value_counts())

test['Year_of_birth_Str'] = [year_categories[x] for x in test['Year_of_birth_Str']]
print(test['Year_of_birth_Str'].value_counts())

1980-1989    16611
1990-2001    15163
1970-1979     4853
1950-1969     1685
Name: Year_of_birth_Str, dtype: int64
1980-1989    7172
1990-2001    6489
1970-1979    2057
1950-1969     778
Name: Year_of_birth_Str, dtype: int64


In [8]:
year_categories1 = {'1982': '1982-2000', '1985': '1982-2000', '1986': '1982-2000', '1987': '1982-2000', '1988': '1982-2000', '1989': '1982-2000',
                    '1990': '1982-2000', '1991': '1982-2000', '1992': '1982-2000', '1993': '1982-2000', 
                   '1994': '1982-2000', '1995': '1982-2000', '1996': '1982-2000', '1997': '1982-2000', 
                   '1998': '1982-2000', '1999': '1982-2000', '2000': '1982-2000', '2001': '2001-2005', 
                   '2002': '2001-2005', '2003': '2001-2005', '2004': '2001-2005', '2005': '2001-2005', 
                   '2006': '2006-2010', '2007': '2006-2010', '2008': '2006-2010', '2009': '2006-2010', 
                   '2010': '2006-2010', '2011': '2011-2015', '2012': '2011-2015', '2013': '2011-2015', 
                   '2014': '2011-2015', '2015': '2011-2015', '2016': '2016-2018', '2017': '2016-2018', 
                   '2018': '2016-2018'}

features['Year_of_recruitment_Str'] = [year_categories1[x] for x in features['Year_of_recruitment_Str']]
print(features['Year_of_recruitment_Str'].value_counts())

test['Year_of_recruitment_Str'] = [year_categories1[x] for x in test['Year_of_recruitment_Str']]
print(test['Year_of_recruitment_Str'].value_counts())

2011-2015    18092
2016-2018    12766
2006-2010     5023
2001-2005     1754
1982-2000      677
Name: Year_of_recruitment_Str, dtype: int64
2011-2015    7744
2016-2018    5498
2006-2010    2196
2001-2005     760
1982-2000     298
Name: Year_of_recruitment_Str, dtype: int64


In [9]:
states = {'ABIA': 'EAST', 'ANAMBRA': 'EAST', 'DELTA': 'EAST', 'BAYELSA': 'EAST',
         'ENUGU': 'EAST', 'EBONYI': 'EAST', 'RIVERS': 'EAST', 'CROSS RIVER': 'EAST',
         'AKWA IBOM': 'EAST', 'IMO': 'EAST', 'KATSINA': 'NORTH', 'KANO': 'NORTH', 'NIGER': 'NORTH',
         'SOKOTO': 'NORTH', 'KANO': 'NORTH', 'KADUNA': 'NORTH', 'BORNO': 'NORTH', 'TARABA':'NORTH',
         'YOBE': 'NORTH', 'ADAMAWA': 'NORTH', 'KEBBI': 'NORTH', 'JIGAWA': 'NORTH', 'ZAMFARA': 'NORTH',
         'KEBBI': 'NORTH', 'PLATEAU': 'NORTH', 'NASSARAWA': 'NORTH', 'FCT': 'NORTH', 'PLATEAU': 'NORTH',
         'BENUE': 'NORTH', 'KOGI':'NORTH', 'BAUCHI': 'NORTH', 'GOMBE': 'NORTH', 'OYO': 'WEST', 'LAGOS': 'WEST', 'OGUN': 'WEST',
         'OSUN': 'WEST', 'EKITI': 'WEST', 'ONDO': 'WEST', 'KWARA': 'WEST', 'EDO': 'WEST'}

features['State_Of_Origin'] = [states[x] for x in features['State_Of_Origin']]
print(features['State_Of_Origin'].value_counts())

test['State_Of_Origin'] = [states[x] for x in test['State_Of_Origin']]
print(test['State_Of_Origin'].value_counts())

WEST     14297
NORTH    13402
EAST     10613
Name: State_Of_Origin, dtype: int64
WEST     6259
NORTH    5684
EAST     4553
Name: State_Of_Origin, dtype: int64


In [10]:
label_count = features['Promoted_or_Not'].value_counts()
label_count

0    35071
1     3241
Name: Promoted_or_Not, dtype: int64

In [11]:
Labels = np.array(features['Promoted_or_Not'])

In [12]:
def encode_string(data):
    enc = preprocessing.LabelEncoder()
    enc.fit(data)
    enc_features = enc.transform(data)
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_features.reshape(-1,1))
    return encoded.transform(enc_features.reshape(-1,1)).toarray()
    
categorical_columns = ['Gender', 'Channel_of_Recruitment', 'State_Of_Origin',
                       'Foreign_schooled', 'Marital_Status', 'Past_Disciplinary_Action', 'Previous_IntraDepartmental_Movement',
                       'Year_of_birth_Str', 'Year_of_recruitment_Str']
Features_enc = encode_string(features['Division'])
for col in categorical_columns:
    temp = encode_string(features[col])
    Features_enc = np.concatenate([Features_enc, temp], axis = 1)
    
print(Features_enc.shape)

test_enc = encode_string(test['Division'])
for col in categorical_columns:
    temps = encode_string(test[col])
    test_enc = np.concatenate([test_enc, temps], axis = 1)
    
print(test_enc.shape)

C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case 

(38312, 35)
(16496, 35)


C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case 

In [13]:
Features_enc = np.concatenate([Features_enc, np.array(features[['Trainings_Attended', 'Last_performance_score',
                                                              'Targets_met', 'Previous_Award', 'Training_score_average']])], axis = 1)

print(Features_enc.shape)

test_enc = np.concatenate([test_enc, np.array(test[['Trainings_Attended', 'Last_performance_score',
                                                              'Targets_met', 'Previous_Award', 'Training_score_average']])], axis = 1)

print(test_enc.shape)

(38312, 40)
(16496, 40)


In [14]:
print(Features_enc[:2, :])
print(test_enc[:2, :])

[[ 0.   1.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   1.   0.
   1.   0.   0.   1.   0.   1.   0.   0.   1.   0.   1.   0.   0.   0.
   1.   0.   0.   0.   0.   1.   0.   2.  12.5  1.   0.  41. ]
 [ 0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   1.   1.   0.   0.
   1.   0.   0.   0.   1.   1.   0.   0.   1.   0.   1.   0.   0.   0.
   0.   1.   0.   0.   0.   1.   0.   2.  12.5  0.   0.  52. ]]
[[ 0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   1.   0.   0.
   0.   1.   0.   0.   1.   1.   0.   0.   1.   0.   1.   0.   0.   1.
   0.   0.   0.   0.   0.   0.   1.   2.   7.5  0.   0.  65. ]
 [ 0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.   0.   1.   0.
   0.   0.   1.   0.   1.   1.   0.   0.   1.   0.   1.   0.   0.   0.
   0.   1.   0.   0.   0.   0.   1.   2.   0.   0.   0.  69. ]]


In [15]:
scaler = preprocessing.StandardScaler().fit(Features_enc[:, 36:])
Features_enc[:, 36:] = scaler.transform(Features_enc[:, 36:])
Features_enc[:, 31:]

array([[ 0.        ,  0.        ,  1.        , ...,  1.35384256,
        -0.15395043, -1.07512768],
       [ 0.        ,  0.        ,  1.        , ..., -0.73863832,
        -0.15395043, -0.25193251],
       [ 0.        ,  0.        ,  1.        , ..., -0.73863832,
        -0.15395043, -1.00029176],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.35384256,
        -0.15395043,  1.16995006],
       [ 0.        ,  0.        ,  0.        , ..., -0.73863832,
        -0.15395043, -1.37447138],
       [ 0.        ,  0.        ,  1.        , ..., -0.73863832,
        -0.15395043, -1.07512768]])

In [16]:
test_enc[:, 36:] = scaler.transform(test_enc[:, 36:])
test_enc[:, 36]

array([-0.05313941, -2.05629819, -0.05313941, ...,  1.28229978,
       -1.3885786 , -0.05313941])

In [17]:
test_enc.shape

(16496, 40)

In [18]:
birth_encoded = encode_string(features['Year_of_birth_Str'])
employers_encoded = encode_string(features['No_of_previous_employers'])

birth_encoded_test = encode_string(test['Year_of_birth_Str'])
employers_encoded_test = encode_string(test['No_of_previous_employers'])

C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case 

In [19]:
Features_enc = np.concatenate([Features_enc, birth_encoded], axis = 1)
print(Features_enc.shape)

test_enc = np.concatenate([test_enc, birth_encoded_test], axis = 1)
print(test_enc.shape)

(38312, 44)
(16496, 44)


In [20]:
employers = {'0': '0-1', '1': '0-1', '2': '2-3', '3': '2-3',
             '4': 'Greater than 3', '5': 'Greater than 3', 'More than 5': 'Greater than 5'}

features['No_of_previous_employers'] = [employers[x] for x in features['No_of_previous_employers']]
test['No_of_previous_employers'] = [employers[x] for x in test['No_of_previous_employers']]

print(features['No_of_previous_employers'].value_counts())
print(test['No_of_previous_employers'].value_counts())

0-1               32139
2-3                3505
Greater than 3     2267
Greater than 5      401
Name: No_of_previous_employers, dtype: int64
0-1               13931
2-3                1469
Greater than 3      940
Greater than 5      156
Name: No_of_previous_employers, dtype: int64


In [21]:
employers_encodedd = encode_string(features['No_of_previous_employers'])
employers_encoded_testt = encode_string(test['No_of_previous_employers'])

C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\TapeCopy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case 

In [22]:
Features_enc = np.concatenate([Features_enc, employers_encodedd], axis = 1)
print(Features_enc.shape)

test_enc = np.concatenate([test_enc, employers_encoded_testt], axis = 1)
print(test_enc.shape)

(38312, 48)
(16496, 48)


In [23]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features_enc.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features_enc[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features_enc[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [24]:
xg_cl1 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.8,
                           reg_lambda = 0.03)

In [25]:
eval_set = [(x_train, y_train), (x_test, y_test)]
xg_cl1.fit(x_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.863699	validation_1-auc:0.850604
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.882769	validation_1-auc:0.881033
[2]	validation_0-auc:0.885067	validation_1-auc:0.884101
[3]	validation_0-auc:0.888418	validation_1-auc:0.889068
[4]	validation_0-auc:0.88888	validation_1-auc:0.88875
[5]	validation_0-auc:0.89105	validation_1-auc:0.889251
[6]	validation_0-auc:0.893914	validation_1-auc:0.891274
[7]	validation_0-auc:0.89391	validation_1-auc:0.893243
[8]	validation_0-auc:0.893892	validation_1-auc:0.893121
[9]	validation_0-auc:0.893736	validation_1-auc:0.892812
[10]	validation_0-auc:0.89383	validation_1-auc:0.893147
[11]	validation_0-auc:0.893662	validation_1-auc:0.893058
[12]	validation_0-auc:0.894148	validation_1-auc:0.893369
[13]	validation_0-auc:0.894052	validation_1-auc:0.893776
[14]	validation_0-auc:0.894234	validation_1-auc:0.893879
[1

[141]	validation_0-auc:0.915736	validation_1-auc:0.905852
[142]	validation_0-auc:0.915869	validation_1-auc:0.905775
[143]	validation_0-auc:0.915886	validation_1-auc:0.905792
[144]	validation_0-auc:0.915984	validation_1-auc:0.905809
[145]	validation_0-auc:0.916024	validation_1-auc:0.905856
[146]	validation_0-auc:0.916058	validation_1-auc:0.905827
[147]	validation_0-auc:0.916063	validation_1-auc:0.905901
[148]	validation_0-auc:0.916157	validation_1-auc:0.905974
[149]	validation_0-auc:0.916236	validation_1-auc:0.906022
[150]	validation_0-auc:0.916287	validation_1-auc:0.905996
[151]	validation_0-auc:0.916343	validation_1-auc:0.905925
[152]	validation_0-auc:0.916465	validation_1-auc:0.905915
[153]	validation_0-auc:0.916594	validation_1-auc:0.905932
[154]	validation_0-auc:0.916624	validation_1-auc:0.905901
[155]	validation_0-auc:0.91674	validation_1-auc:0.905772
[156]	validation_0-auc:0.916841	validation_1-auc:0.905781
[157]	validation_0-auc:0.916905	validation_1-auc:0.905703
[158]	validatio

[283]	validation_0-auc:0.928464	validation_1-auc:0.909602
[284]	validation_0-auc:0.928564	validation_1-auc:0.909546
[285]	validation_0-auc:0.928619	validation_1-auc:0.909526
[286]	validation_0-auc:0.928665	validation_1-auc:0.909492
[287]	validation_0-auc:0.928732	validation_1-auc:0.909448
[288]	validation_0-auc:0.92888	validation_1-auc:0.909559
[289]	validation_0-auc:0.929024	validation_1-auc:0.909625
[290]	validation_0-auc:0.929081	validation_1-auc:0.909572
[291]	validation_0-auc:0.929141	validation_1-auc:0.909599
[292]	validation_0-auc:0.929207	validation_1-auc:0.90962
[293]	validation_0-auc:0.929302	validation_1-auc:0.909703
[294]	validation_0-auc:0.929355	validation_1-auc:0.90977
[295]	validation_0-auc:0.929423	validation_1-auc:0.909781
[296]	validation_0-auc:0.929562	validation_1-auc:0.909774
[297]	validation_0-auc:0.929614	validation_1-auc:0.90976
[298]	validation_0-auc:0.929887	validation_1-auc:0.909785
[299]	validation_0-auc:0.930005	validation_1-auc:0.90981
[300]	validation_0-

[425]	validation_0-auc:0.941022	validation_1-auc:0.910578
[426]	validation_0-auc:0.941063	validation_1-auc:0.910585
[427]	validation_0-auc:0.941199	validation_1-auc:0.910551
[428]	validation_0-auc:0.941318	validation_1-auc:0.910558
[429]	validation_0-auc:0.941383	validation_1-auc:0.910581
[430]	validation_0-auc:0.94146	validation_1-auc:0.910558
[431]	validation_0-auc:0.941528	validation_1-auc:0.910541
[432]	validation_0-auc:0.941698	validation_1-auc:0.910622
[433]	validation_0-auc:0.941772	validation_1-auc:0.910662
[434]	validation_0-auc:0.941841	validation_1-auc:0.910671
[435]	validation_0-auc:0.941949	validation_1-auc:0.910661
[436]	validation_0-auc:0.942107	validation_1-auc:0.910698
[437]	validation_0-auc:0.942157	validation_1-auc:0.910691
[438]	validation_0-auc:0.942195	validation_1-auc:0.910674
[439]	validation_0-auc:0.942295	validation_1-auc:0.910687
[440]	validation_0-auc:0.942358	validation_1-auc:0.910649
[441]	validation_0-auc:0.942423	validation_1-auc:0.910611
[442]	validatio

[567]	validation_0-auc:0.950437	validation_1-auc:0.910478
[568]	validation_0-auc:0.950445	validation_1-auc:0.910523
[569]	validation_0-auc:0.950458	validation_1-auc:0.910505
[570]	validation_0-auc:0.950502	validation_1-auc:0.910484
[571]	validation_0-auc:0.950516	validation_1-auc:0.910467
[572]	validation_0-auc:0.950598	validation_1-auc:0.910478
[573]	validation_0-auc:0.950643	validation_1-auc:0.910531
[574]	validation_0-auc:0.950694	validation_1-auc:0.910515
[575]	validation_0-auc:0.95074	validation_1-auc:0.910534
[576]	validation_0-auc:0.950788	validation_1-auc:0.910521
[577]	validation_0-auc:0.950809	validation_1-auc:0.910549
[578]	validation_0-auc:0.95089	validation_1-auc:0.910542
[579]	validation_0-auc:0.950936	validation_1-auc:0.910566
[580]	validation_0-auc:0.951008	validation_1-auc:0.910591
[581]	validation_0-auc:0.951052	validation_1-auc:0.91059
[582]	validation_0-auc:0.951129	validation_1-auc:0.910582
[583]	validation_0-auc:0.951179	validation_1-auc:0.910566
[584]	validation_

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.03, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.8, verbosity=1)

In [26]:
preds1 = xg_cl1.predict(test_enc)

In [27]:
sample1 = pd.read_csv('sample_submission2.csv')

In [28]:
sample1.EmployeeNo = test.EmployeeNo

In [29]:
sample1.Promoted_or_Not = preds1

In [30]:
sample1.to_csv('intercampus1.csv', index = False)

In [42]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


    
print_metrics(y_test, preds2)    

                 Confusion matrix
                 Score positive    Score negative
Actual positive      7016                13
Actual negative       402               232

Accuracy  0.95
 
           Positive      Negative
Num case     7029           634
Precision    0.95          0.95
Recall       1.00          0.37
F1           0.97          0.53


In [31]:
xg_cl2 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.9,
                           reg_lambda = 0.04)

In [32]:
eval_set = [(x_train, y_train), (x_test, y_test)]
xg_cl2.fit(x_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.866271	validation_1-auc:0.860797
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.877947	validation_1-auc:0.871124
[2]	validation_0-auc:0.880222	validation_1-auc:0.876851
[3]	validation_0-auc:0.881135	validation_1-auc:0.877282
[4]	validation_0-auc:0.8811	validation_1-auc:0.877313
[5]	validation_0-auc:0.881645	validation_1-auc:0.878067
[6]	validation_0-auc:0.881864	validation_1-auc:0.878579
[7]	validation_0-auc:0.881518	validation_1-auc:0.878909
[8]	validation_0-auc:0.882003	validation_1-auc:0.878924
[9]	validation_0-auc:0.887852	validation_1-auc:0.886067
[10]	validation_0-auc:0.887025	validation_1-auc:0.885998
[11]	validation_0-auc:0.886372	validation_1-auc:0.884918
[12]	validation_0-auc:0.886488	validation_1-auc:0.885151
[13]	validation_0-auc:0.889922	validation_1-auc:0.889511
[14]	validation_0-auc:0.889768	validation_1-auc:0.889617

[141]	validation_0-auc:0.914901	validation_1-auc:0.904694
[142]	validation_0-auc:0.914978	validation_1-auc:0.904727
[143]	validation_0-auc:0.915039	validation_1-auc:0.904782
[144]	validation_0-auc:0.915274	validation_1-auc:0.904712
[145]	validation_0-auc:0.915302	validation_1-auc:0.904738
[146]	validation_0-auc:0.915317	validation_1-auc:0.904772
[147]	validation_0-auc:0.915375	validation_1-auc:0.904762
[148]	validation_0-auc:0.915526	validation_1-auc:0.9047
[149]	validation_0-auc:0.915637	validation_1-auc:0.904733
[150]	validation_0-auc:0.915725	validation_1-auc:0.904861
[151]	validation_0-auc:0.915762	validation_1-auc:0.904987
[152]	validation_0-auc:0.915796	validation_1-auc:0.905009
[153]	validation_0-auc:0.915888	validation_1-auc:0.905054
[154]	validation_0-auc:0.915924	validation_1-auc:0.905028
[155]	validation_0-auc:0.915929	validation_1-auc:0.905017
[156]	validation_0-auc:0.916016	validation_1-auc:0.90509
[157]	validation_0-auc:0.916053	validation_1-auc:0.905166
[158]	validation_

[283]	validation_0-auc:0.927033	validation_1-auc:0.908972
[284]	validation_0-auc:0.927116	validation_1-auc:0.908967
[285]	validation_0-auc:0.927167	validation_1-auc:0.909036
[286]	validation_0-auc:0.927255	validation_1-auc:0.909037
[287]	validation_0-auc:0.927287	validation_1-auc:0.909046
[288]	validation_0-auc:0.927415	validation_1-auc:0.909083
[289]	validation_0-auc:0.927474	validation_1-auc:0.9091
[290]	validation_0-auc:0.927587	validation_1-auc:0.909111
[291]	validation_0-auc:0.927905	validation_1-auc:0.909096
[292]	validation_0-auc:0.92798	validation_1-auc:0.909136
[293]	validation_0-auc:0.928072	validation_1-auc:0.909109
[294]	validation_0-auc:0.928138	validation_1-auc:0.909128
[295]	validation_0-auc:0.92826	validation_1-auc:0.90919
[296]	validation_0-auc:0.928325	validation_1-auc:0.909261
[297]	validation_0-auc:0.928384	validation_1-auc:0.909299
[298]	validation_0-auc:0.928537	validation_1-auc:0.909355
[299]	validation_0-auc:0.928608	validation_1-auc:0.909308
[300]	validation_0-

[425]	validation_0-auc:0.938813	validation_1-auc:0.910393
[426]	validation_0-auc:0.938861	validation_1-auc:0.910376
[427]	validation_0-auc:0.938986	validation_1-auc:0.910409
[428]	validation_0-auc:0.939134	validation_1-auc:0.910445
[429]	validation_0-auc:0.939195	validation_1-auc:0.910476
[430]	validation_0-auc:0.939357	validation_1-auc:0.910607
[431]	validation_0-auc:0.939417	validation_1-auc:0.910539
[432]	validation_0-auc:0.939512	validation_1-auc:0.910579
[433]	validation_0-auc:0.939578	validation_1-auc:0.910576
[434]	validation_0-auc:0.939662	validation_1-auc:0.910597
[435]	validation_0-auc:0.939739	validation_1-auc:0.910613
[436]	validation_0-auc:0.939794	validation_1-auc:0.910611
[437]	validation_0-auc:0.93985	validation_1-auc:0.910611
[438]	validation_0-auc:0.939868	validation_1-auc:0.910619
[439]	validation_0-auc:0.93998	validation_1-auc:0.910612
[440]	validation_0-auc:0.94005	validation_1-auc:0.910615
[441]	validation_0-auc:0.940138	validation_1-auc:0.910629
[442]	validation_

[567]	validation_0-auc:0.948142	validation_1-auc:0.911004
[568]	validation_0-auc:0.948159	validation_1-auc:0.910999
[569]	validation_0-auc:0.948181	validation_1-auc:0.911
[570]	validation_0-auc:0.94822	validation_1-auc:0.910992
[571]	validation_0-auc:0.948243	validation_1-auc:0.910994
[572]	validation_0-auc:0.948275	validation_1-auc:0.911002
[573]	validation_0-auc:0.948343	validation_1-auc:0.910983
[574]	validation_0-auc:0.948357	validation_1-auc:0.910992
[575]	validation_0-auc:0.948419	validation_1-auc:0.910931
[576]	validation_0-auc:0.948498	validation_1-auc:0.910959
[577]	validation_0-auc:0.948523	validation_1-auc:0.911005
[578]	validation_0-auc:0.948593	validation_1-auc:0.911005
[579]	validation_0-auc:0.948649	validation_1-auc:0.91103
[580]	validation_0-auc:0.948692	validation_1-auc:0.911001
[581]	validation_0-auc:0.948717	validation_1-auc:0.911012
[582]	validation_0-auc:0.948756	validation_1-auc:0.911028
[583]	validation_0-auc:0.948789	validation_1-auc:0.911039
[584]	validation_0-

[709]	validation_0-auc:0.955017	validation_1-auc:0.910539
[710]	validation_0-auc:0.955048	validation_1-auc:0.910547
[711]	validation_0-auc:0.955077	validation_1-auc:0.910569
[712]	validation_0-auc:0.955145	validation_1-auc:0.910561
[713]	validation_0-auc:0.95516	validation_1-auc:0.910563
[714]	validation_0-auc:0.955185	validation_1-auc:0.910573
[715]	validation_0-auc:0.955219	validation_1-auc:0.910571
[716]	validation_0-auc:0.955325	validation_1-auc:0.91056
[717]	validation_0-auc:0.955352	validation_1-auc:0.910568
[718]	validation_0-auc:0.955362	validation_1-auc:0.910593
[719]	validation_0-auc:0.955441	validation_1-auc:0.910582
[720]	validation_0-auc:0.955524	validation_1-auc:0.910587
[721]	validation_0-auc:0.955603	validation_1-auc:0.910568
[722]	validation_0-auc:0.955713	validation_1-auc:0.910574
[723]	validation_0-auc:0.955803	validation_1-auc:0.910582
[724]	validation_0-auc:0.955847	validation_1-auc:0.910577
[725]	validation_0-auc:0.955979	validation_1-auc:0.910558
[726]	validation

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.04, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.9, verbosity=1)

In [33]:
preds2 = xg_cl1.predict(test_enc)

In [34]:
sample2 = pd.read_csv('sample_submission2.csv')

In [35]:
sample2.EmployeeNo = test.EmployeeNo

In [36]:
sample2.Promoted_or_Not = preds2

In [37]:
sample2.to_csv('intercampus2.csv', index = False)

In [38]:
xg_cl3 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.9,
                           reg_lambda = 0.02)

In [39]:
eval_set = [(x_train, y_train), (x_test, y_test)]
xg_cl3.fit(x_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.866284	validation_1-auc:0.860822
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.878052	validation_1-auc:0.870964
[2]	validation_0-auc:0.880194	validation_1-auc:0.876332
[3]	validation_0-auc:0.881331	validation_1-auc:0.876746
[4]	validation_0-auc:0.881359	validation_1-auc:0.877413
[5]	validation_0-auc:0.881618	validation_1-auc:0.877824
[6]	validation_0-auc:0.882041	validation_1-auc:0.878345
[7]	validation_0-auc:0.881514	validation_1-auc:0.878511
[8]	validation_0-auc:0.881997	validation_1-auc:0.879332
[9]	validation_0-auc:0.88795	validation_1-auc:0.886167
[10]	validation_0-auc:0.887088	validation_1-auc:0.886064
[11]	validation_0-auc:0.886533	validation_1-auc:0.885159
[12]	validation_0-auc:0.886664	validation_1-auc:0.885819
[13]	validation_0-auc:0.890139	validation_1-auc:0.889983
[14]	validation_0-auc:0.889997	validation_1-auc:0.88990

[141]	validation_0-auc:0.91481	validation_1-auc:0.90499
[142]	validation_0-auc:0.914894	validation_1-auc:0.904993
[143]	validation_0-auc:0.91496	validation_1-auc:0.905069
[144]	validation_0-auc:0.915139	validation_1-auc:0.904901
[145]	validation_0-auc:0.915226	validation_1-auc:0.904901
[146]	validation_0-auc:0.915256	validation_1-auc:0.904985
[147]	validation_0-auc:0.915307	validation_1-auc:0.905013
[148]	validation_0-auc:0.915525	validation_1-auc:0.904929
[149]	validation_0-auc:0.915637	validation_1-auc:0.905001
[150]	validation_0-auc:0.915757	validation_1-auc:0.90505
[151]	validation_0-auc:0.915792	validation_1-auc:0.905161
[152]	validation_0-auc:0.915832	validation_1-auc:0.905185
[153]	validation_0-auc:0.91594	validation_1-auc:0.905184
[154]	validation_0-auc:0.916047	validation_1-auc:0.90514
[155]	validation_0-auc:0.916057	validation_1-auc:0.905131
[156]	validation_0-auc:0.916111	validation_1-auc:0.905202
[157]	validation_0-auc:0.916159	validation_1-auc:0.905273
[158]	validation_0-a

[283]	validation_0-auc:0.927155	validation_1-auc:0.909163
[284]	validation_0-auc:0.927231	validation_1-auc:0.909134
[285]	validation_0-auc:0.92729	validation_1-auc:0.909193
[286]	validation_0-auc:0.927378	validation_1-auc:0.909194
[287]	validation_0-auc:0.927424	validation_1-auc:0.909197
[288]	validation_0-auc:0.927556	validation_1-auc:0.909253
[289]	validation_0-auc:0.927608	validation_1-auc:0.909283
[290]	validation_0-auc:0.927723	validation_1-auc:0.909296
[291]	validation_0-auc:0.928031	validation_1-auc:0.909299
[292]	validation_0-auc:0.928102	validation_1-auc:0.909314
[293]	validation_0-auc:0.928194	validation_1-auc:0.909311
[294]	validation_0-auc:0.928249	validation_1-auc:0.909322
[295]	validation_0-auc:0.928321	validation_1-auc:0.909296
[296]	validation_0-auc:0.928389	validation_1-auc:0.909377
[297]	validation_0-auc:0.928453	validation_1-auc:0.909405
[298]	validation_0-auc:0.928626	validation_1-auc:0.909493
[299]	validation_0-auc:0.928674	validation_1-auc:0.909522
[300]	validatio

[425]	validation_0-auc:0.938671	validation_1-auc:0.910384
[426]	validation_0-auc:0.938733	validation_1-auc:0.91035
[427]	validation_0-auc:0.938858	validation_1-auc:0.910361
[428]	validation_0-auc:0.938996	validation_1-auc:0.910382
[429]	validation_0-auc:0.939043	validation_1-auc:0.910382
[430]	validation_0-auc:0.939222	validation_1-auc:0.910483
[431]	validation_0-auc:0.939276	validation_1-auc:0.910415
[432]	validation_0-auc:0.939385	validation_1-auc:0.910442
[433]	validation_0-auc:0.939456	validation_1-auc:0.910471
[434]	validation_0-auc:0.939538	validation_1-auc:0.910493
[435]	validation_0-auc:0.939615	validation_1-auc:0.910502
[436]	validation_0-auc:0.939666	validation_1-auc:0.910507
[437]	validation_0-auc:0.939733	validation_1-auc:0.910534
[438]	validation_0-auc:0.939751	validation_1-auc:0.910527
[439]	validation_0-auc:0.939878	validation_1-auc:0.910553
[440]	validation_0-auc:0.939944	validation_1-auc:0.910571
[441]	validation_0-auc:0.940008	validation_1-auc:0.91053
[442]	validation

[567]	validation_0-auc:0.948233	validation_1-auc:0.911036
[568]	validation_0-auc:0.948249	validation_1-auc:0.911043
[569]	validation_0-auc:0.948268	validation_1-auc:0.91105
[570]	validation_0-auc:0.94834	validation_1-auc:0.911026
[571]	validation_0-auc:0.948352	validation_1-auc:0.911024
[572]	validation_0-auc:0.948401	validation_1-auc:0.911021
[573]	validation_0-auc:0.948467	validation_1-auc:0.910998
[574]	validation_0-auc:0.948486	validation_1-auc:0.911013
[575]	validation_0-auc:0.948573	validation_1-auc:0.910959
[576]	validation_0-auc:0.948656	validation_1-auc:0.910998
[577]	validation_0-auc:0.948676	validation_1-auc:0.911037
[578]	validation_0-auc:0.948739	validation_1-auc:0.911054
[579]	validation_0-auc:0.948796	validation_1-auc:0.911065
[580]	validation_0-auc:0.948838	validation_1-auc:0.911042
[581]	validation_0-auc:0.948862	validation_1-auc:0.911071
[582]	validation_0-auc:0.948915	validation_1-auc:0.911084
[583]	validation_0-auc:0.948957	validation_1-auc:0.911092
[584]	validation

[709]	validation_0-auc:0.955082	validation_1-auc:0.910661
[710]	validation_0-auc:0.955124	validation_1-auc:0.910647
[711]	validation_0-auc:0.955151	validation_1-auc:0.910662
[712]	validation_0-auc:0.955223	validation_1-auc:0.910667
[713]	validation_0-auc:0.955237	validation_1-auc:0.910668
[714]	validation_0-auc:0.955265	validation_1-auc:0.910667
[715]	validation_0-auc:0.955298	validation_1-auc:0.910665
[716]	validation_0-auc:0.955404	validation_1-auc:0.910657
[717]	validation_0-auc:0.955425	validation_1-auc:0.910666
[718]	validation_0-auc:0.955435	validation_1-auc:0.910678
[719]	validation_0-auc:0.955514	validation_1-auc:0.910674
[720]	validation_0-auc:0.955547	validation_1-auc:0.910695
[721]	validation_0-auc:0.955621	validation_1-auc:0.91067
[722]	validation_0-auc:0.955731	validation_1-auc:0.910675
[723]	validation_0-auc:0.955823	validation_1-auc:0.91069
[724]	validation_0-auc:0.955863	validation_1-auc:0.910685
[725]	validation_0-auc:0.95599	validation_1-auc:0.910681
[726]	validation_

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.02, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.9, verbosity=1)

In [40]:
preds3 = xg_cl3.predict(test_enc)

In [41]:
sample3 = pd.read_csv('sample_submission2.csv')

In [42]:
sample3.EmployeeNo = test.EmployeeNo

In [43]:
sample3.Promoted_or_Not = preds3

In [44]:
sample3.to_csv('intercampus3.csv', index = False)

In [45]:
preds4 = xg_cl2.predict(test_enc)

In [46]:
sample4 = pd.read_csv('sample_submission2.csv')

In [47]:
sample4.EmployeeNo = test.EmployeeNo

In [48]:
sample4.Promoted_or_Not = preds4

In [49]:
sample4.to_csv('intercampus4.csv', index = False)

In [50]:
xg_cl5 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.9,
                           reg_lambda = 0.03)

In [51]:
eval_set = [(x_train, y_train), (x_test, y_test)]
xg_cl5.fit(x_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.866288	validation_1-auc:0.860837
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.877972	validation_1-auc:0.871066
[2]	validation_0-auc:0.880231	validation_1-auc:0.876394
[3]	validation_0-auc:0.881292	validation_1-auc:0.876765
[4]	validation_0-auc:0.881298	validation_1-auc:0.877356
[5]	validation_0-auc:0.881684	validation_1-auc:0.877779
[6]	validation_0-auc:0.882022	validation_1-auc:0.878336
[7]	validation_0-auc:0.881582	validation_1-auc:0.878608
[8]	validation_0-auc:0.882026	validation_1-auc:0.879448
[9]	validation_0-auc:0.887925	validation_1-auc:0.886157
[10]	validation_0-auc:0.887092	validation_1-auc:0.886156
[11]	validation_0-auc:0.886456	validation_1-auc:0.884967
[12]	validation_0-auc:0.886564	validation_1-auc:0.885682
[13]	validation_0-auc:0.890089	validation_1-auc:0.889837
[14]	validation_0-auc:0.88994	validation_1-auc:0.88955

[142]	validation_0-auc:0.914949	validation_1-auc:0.904923
[143]	validation_0-auc:0.915031	validation_1-auc:0.904974
[144]	validation_0-auc:0.915203	validation_1-auc:0.904894
[145]	validation_0-auc:0.91525	validation_1-auc:0.904875
[146]	validation_0-auc:0.915277	validation_1-auc:0.904983
[147]	validation_0-auc:0.915322	validation_1-auc:0.904948
[148]	validation_0-auc:0.915509	validation_1-auc:0.904853
[149]	validation_0-auc:0.915628	validation_1-auc:0.904892
[150]	validation_0-auc:0.915755	validation_1-auc:0.904998
[151]	validation_0-auc:0.915816	validation_1-auc:0.905096
[152]	validation_0-auc:0.915837	validation_1-auc:0.905107
[153]	validation_0-auc:0.915936	validation_1-auc:0.905206
[154]	validation_0-auc:0.916023	validation_1-auc:0.905024
[155]	validation_0-auc:0.916037	validation_1-auc:0.905082
[156]	validation_0-auc:0.916055	validation_1-auc:0.9051
[157]	validation_0-auc:0.916123	validation_1-auc:0.905121
[158]	validation_0-auc:0.916147	validation_1-auc:0.905128
[159]	validation_

[284]	validation_0-auc:0.927043	validation_1-auc:0.908979
[285]	validation_0-auc:0.927106	validation_1-auc:0.909028
[286]	validation_0-auc:0.92719	validation_1-auc:0.909004
[287]	validation_0-auc:0.927234	validation_1-auc:0.909006
[288]	validation_0-auc:0.927375	validation_1-auc:0.909027
[289]	validation_0-auc:0.927427	validation_1-auc:0.909037
[290]	validation_0-auc:0.927547	validation_1-auc:0.909094
[291]	validation_0-auc:0.927837	validation_1-auc:0.909123
[292]	validation_0-auc:0.927917	validation_1-auc:0.909151
[293]	validation_0-auc:0.928007	validation_1-auc:0.909146
[294]	validation_0-auc:0.928064	validation_1-auc:0.909177
[295]	validation_0-auc:0.928184	validation_1-auc:0.90922
[296]	validation_0-auc:0.928242	validation_1-auc:0.909307
[297]	validation_0-auc:0.928304	validation_1-auc:0.909338
[298]	validation_0-auc:0.928476	validation_1-auc:0.90938
[299]	validation_0-auc:0.928551	validation_1-auc:0.909326
[300]	validation_0-auc:0.928665	validation_1-auc:0.909291
[301]	validation_

[426]	validation_0-auc:0.938728	validation_1-auc:0.910363
[427]	validation_0-auc:0.938854	validation_1-auc:0.910385
[428]	validation_0-auc:0.938951	validation_1-auc:0.910394
[429]	validation_0-auc:0.939004	validation_1-auc:0.910418
[430]	validation_0-auc:0.939172	validation_1-auc:0.9105
[431]	validation_0-auc:0.939231	validation_1-auc:0.910448
[432]	validation_0-auc:0.939331	validation_1-auc:0.910497
[433]	validation_0-auc:0.939386	validation_1-auc:0.910518
[434]	validation_0-auc:0.939458	validation_1-auc:0.910535
[435]	validation_0-auc:0.939547	validation_1-auc:0.910545
[436]	validation_0-auc:0.939603	validation_1-auc:0.910551
[437]	validation_0-auc:0.939685	validation_1-auc:0.910567
[438]	validation_0-auc:0.939704	validation_1-auc:0.910568
[439]	validation_0-auc:0.939826	validation_1-auc:0.910579
[440]	validation_0-auc:0.939895	validation_1-auc:0.910568
[441]	validation_0-auc:0.939976	validation_1-auc:0.910577
[442]	validation_0-auc:0.940016	validation_1-auc:0.910615
[443]	validation

[568]	validation_0-auc:0.948221	validation_1-auc:0.910845
[569]	validation_0-auc:0.948243	validation_1-auc:0.910831
[570]	validation_0-auc:0.948293	validation_1-auc:0.910841
[571]	validation_0-auc:0.948318	validation_1-auc:0.910874
[572]	validation_0-auc:0.948348	validation_1-auc:0.910858
[573]	validation_0-auc:0.948411	validation_1-auc:0.910862
[574]	validation_0-auc:0.94843	validation_1-auc:0.910877
[575]	validation_0-auc:0.94851	validation_1-auc:0.91082
[576]	validation_0-auc:0.948586	validation_1-auc:0.910784
[577]	validation_0-auc:0.948611	validation_1-auc:0.910822
[578]	validation_0-auc:0.948684	validation_1-auc:0.910847
[579]	validation_0-auc:0.94874	validation_1-auc:0.910856
[580]	validation_0-auc:0.948781	validation_1-auc:0.910838
[581]	validation_0-auc:0.948803	validation_1-auc:0.910842
[582]	validation_0-auc:0.948836	validation_1-auc:0.910847
[583]	validation_0-auc:0.948867	validation_1-auc:0.910848
[584]	validation_0-auc:0.948933	validation_1-auc:0.910826
[585]	validation_0

[710]	validation_0-auc:0.955034	validation_1-auc:0.910419
[711]	validation_0-auc:0.955065	validation_1-auc:0.910418
[712]	validation_0-auc:0.955134	validation_1-auc:0.910418
[713]	validation_0-auc:0.955148	validation_1-auc:0.910416
[714]	validation_0-auc:0.955172	validation_1-auc:0.910432
[715]	validation_0-auc:0.955201	validation_1-auc:0.910425
[716]	validation_0-auc:0.955307	validation_1-auc:0.910415
[717]	validation_0-auc:0.955342	validation_1-auc:0.910408
[718]	validation_0-auc:0.955354	validation_1-auc:0.91042
[719]	validation_0-auc:0.955425	validation_1-auc:0.910414
[720]	validation_0-auc:0.95552	validation_1-auc:0.910423
[721]	validation_0-auc:0.955596	validation_1-auc:0.910398
[722]	validation_0-auc:0.955724	validation_1-auc:0.910396
[723]	validation_0-auc:0.955813	validation_1-auc:0.91037
[724]	validation_0-auc:0.955861	validation_1-auc:0.910355
[725]	validation_0-auc:0.95598	validation_1-auc:0.910337
[726]	validation_0-auc:0.956006	validation_1-auc:0.910328
[727]	validation_0

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.03, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.9, verbosity=1)

In [52]:
preds5 = xg_cl5.predict(test_enc)

In [53]:
sample5 = pd.read_csv('sample_submission2.csv')

In [54]:
preds5 = xg_cl5.predict(test_enc)

In [55]:
sample5.EmployeeNo = test.EmployeeNo

In [56]:
sample5.Promoted_or_Not = preds5

In [57]:
sample5.to_csv('intercampus5.csv', index = False)

In [58]:
xg_cl6 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.9,
                           reg_lambda = 0.03, gamma = 1)

In [59]:
eval_set = [(x_train, y_train), (x_test, y_test)]
xg_cl6.fit(x_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.865679	validation_1-auc:0.8609
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.877435	validation_1-auc:0.870454
[2]	validation_0-auc:0.879757	validation_1-auc:0.873974
[3]	validation_0-auc:0.880641	validation_1-auc:0.876416
[4]	validation_0-auc:0.880776	validation_1-auc:0.876143
[5]	validation_0-auc:0.88123	validation_1-auc:0.876495
[6]	validation_0-auc:0.881577	validation_1-auc:0.877243
[7]	validation_0-auc:0.881101	validation_1-auc:0.877464
[8]	validation_0-auc:0.881709	validation_1-auc:0.878174
[9]	validation_0-auc:0.887559	validation_1-auc:0.885933
[10]	validation_0-auc:0.886797	validation_1-auc:0.885549
[11]	validation_0-auc:0.886113	validation_1-auc:0.884653
[12]	validation_0-auc:0.886249	validation_1-auc:0.88457
[13]	validation_0-auc:0.889712	validation_1-auc:0.888896
[14]	validation_0-auc:0.889632	validation_1-auc:0.8891
[15

[141]	validation_0-auc:0.91448	validation_1-auc:0.905079
[142]	validation_0-auc:0.914555	validation_1-auc:0.905053
[143]	validation_0-auc:0.914623	validation_1-auc:0.905172
[144]	validation_0-auc:0.914822	validation_1-auc:0.905007
[145]	validation_0-auc:0.914868	validation_1-auc:0.905052
[146]	validation_0-auc:0.914874	validation_1-auc:0.90509
[147]	validation_0-auc:0.914924	validation_1-auc:0.9051
[148]	validation_0-auc:0.915083	validation_1-auc:0.90508
[149]	validation_0-auc:0.915119	validation_1-auc:0.905122
[150]	validation_0-auc:0.915204	validation_1-auc:0.905209
[151]	validation_0-auc:0.915282	validation_1-auc:0.905295
[152]	validation_0-auc:0.915288	validation_1-auc:0.905298
[153]	validation_0-auc:0.915395	validation_1-auc:0.905382
[154]	validation_0-auc:0.915506	validation_1-auc:0.905302
[155]	validation_0-auc:0.915515	validation_1-auc:0.905342
[156]	validation_0-auc:0.915542	validation_1-auc:0.905378
[157]	validation_0-auc:0.915635	validation_1-auc:0.905487
[158]	validation_0-

[283]	validation_0-auc:0.926376	validation_1-auc:0.909106
[284]	validation_0-auc:0.926453	validation_1-auc:0.909082
[285]	validation_0-auc:0.926516	validation_1-auc:0.909184
[286]	validation_0-auc:0.926594	validation_1-auc:0.909275
[287]	validation_0-auc:0.92663	validation_1-auc:0.909266
[288]	validation_0-auc:0.926821	validation_1-auc:0.909269
[289]	validation_0-auc:0.926884	validation_1-auc:0.909281
[290]	validation_0-auc:0.926996	validation_1-auc:0.909296
[291]	validation_0-auc:0.927083	validation_1-auc:0.909295
[292]	validation_0-auc:0.927149	validation_1-auc:0.909351
[293]	validation_0-auc:0.92725	validation_1-auc:0.909344
[294]	validation_0-auc:0.927307	validation_1-auc:0.909351
[295]	validation_0-auc:0.927372	validation_1-auc:0.909326
[296]	validation_0-auc:0.927431	validation_1-auc:0.909376
[297]	validation_0-auc:0.927484	validation_1-auc:0.909384
[298]	validation_0-auc:0.927648	validation_1-auc:0.909434
[299]	validation_0-auc:0.92772	validation_1-auc:0.909398
[300]	validation_

[425]	validation_0-auc:0.937534	validation_1-auc:0.910496
[426]	validation_0-auc:0.937588	validation_1-auc:0.910451
[427]	validation_0-auc:0.937717	validation_1-auc:0.910469
[428]	validation_0-auc:0.937819	validation_1-auc:0.91046
[429]	validation_0-auc:0.937869	validation_1-auc:0.910484
[430]	validation_0-auc:0.938045	validation_1-auc:0.910582
[431]	validation_0-auc:0.938103	validation_1-auc:0.910584
[432]	validation_0-auc:0.938215	validation_1-auc:0.910631
[433]	validation_0-auc:0.938279	validation_1-auc:0.910662
[434]	validation_0-auc:0.938349	validation_1-auc:0.910683
[435]	validation_0-auc:0.938431	validation_1-auc:0.910679
[436]	validation_0-auc:0.938489	validation_1-auc:0.910709
[437]	validation_0-auc:0.938565	validation_1-auc:0.910732
[438]	validation_0-auc:0.938586	validation_1-auc:0.910733
[439]	validation_0-auc:0.938715	validation_1-auc:0.91074
[440]	validation_0-auc:0.938793	validation_1-auc:0.910745
[441]	validation_0-auc:0.938862	validation_1-auc:0.910716
[442]	validation

[567]	validation_0-auc:0.947155	validation_1-auc:0.911145
[568]	validation_0-auc:0.947163	validation_1-auc:0.911164
[569]	validation_0-auc:0.947188	validation_1-auc:0.911154
[570]	validation_0-auc:0.947257	validation_1-auc:0.911144
[571]	validation_0-auc:0.947268	validation_1-auc:0.911157
[572]	validation_0-auc:0.94729	validation_1-auc:0.911166
[573]	validation_0-auc:0.947355	validation_1-auc:0.911179
[574]	validation_0-auc:0.94736	validation_1-auc:0.911208
[575]	validation_0-auc:0.947442	validation_1-auc:0.911149
[576]	validation_0-auc:0.947519	validation_1-auc:0.911162
[577]	validation_0-auc:0.947548	validation_1-auc:0.911201
[578]	validation_0-auc:0.947621	validation_1-auc:0.911215
[579]	validation_0-auc:0.947675	validation_1-auc:0.911248
[580]	validation_0-auc:0.947723	validation_1-auc:0.911213
[581]	validation_0-auc:0.947748	validation_1-auc:0.911222
[582]	validation_0-auc:0.947795	validation_1-auc:0.911208
[583]	validation_0-auc:0.947827	validation_1-auc:0.911201
[584]	validation

[709]	validation_0-auc:0.953609	validation_1-auc:0.910491
[710]	validation_0-auc:0.953652	validation_1-auc:0.910479
[711]	validation_0-auc:0.953694	validation_1-auc:0.910474
[712]	validation_0-auc:0.953767	validation_1-auc:0.91049
[713]	validation_0-auc:0.95378	validation_1-auc:0.910486
[714]	validation_0-auc:0.953806	validation_1-auc:0.910503
[715]	validation_0-auc:0.953835	validation_1-auc:0.910493
[716]	validation_0-auc:0.95394	validation_1-auc:0.910501
[717]	validation_0-auc:0.95395	validation_1-auc:0.910514
[718]	validation_0-auc:0.953966	validation_1-auc:0.910523
[719]	validation_0-auc:0.95403	validation_1-auc:0.910522
[720]	validation_0-auc:0.954055	validation_1-auc:0.910534
[721]	validation_0-auc:0.954111	validation_1-auc:0.910544
[722]	validation_0-auc:0.954226	validation_1-auc:0.910543
[723]	validation_0-auc:0.954306	validation_1-auc:0.910548
[724]	validation_0-auc:0.954352	validation_1-auc:0.910549
[725]	validation_0-auc:0.954488	validation_1-auc:0.910536
[726]	validation_0-

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.03, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.9, verbosity=1)

In [60]:
preds6 = xg_cl6.predict(test_enc)

In [61]:
sample6 = pd.read_csv('sample_submission2.csv')

In [62]:
sample6.EmployeeNo = test.EmployeeNo

In [63]:
sample6.Promoted_or_Not = preds6

In [64]:
sample6.to_csv('intercampus6.csv', index = False)

In [65]:
xg_cl7 = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.7, subsample = 0.9,
                           reg_lambda = 0.03)

In [66]:
eval_set = [(Features_enc, Labels), (x_train, y_train)]
xg_cl7.fit(Features_enc, Labels, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-auc:0.877181	validation_1-auc:0.87536
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.881543	validation_1-auc:0.878657
[2]	validation_0-auc:0.882266	validation_1-auc:0.879285
[3]	validation_0-auc:0.881943	validation_1-auc:0.878675
[4]	validation_0-auc:0.883016	validation_1-auc:0.879937
[5]	validation_0-auc:0.882949	validation_1-auc:0.879941
[6]	validation_0-auc:0.88282	validation_1-auc:0.87971
[7]	validation_0-auc:0.88256	validation_1-auc:0.879319
[8]	validation_0-auc:0.882347	validation_1-auc:0.879091
[9]	validation_0-auc:0.882316	validation_1-auc:0.879195
[10]	validation_0-auc:0.882734	validation_1-auc:0.87956
[11]	validation_0-auc:0.886654	validation_1-auc:0.883255
[12]	validation_0-auc:0.886372	validation_1-auc:0.882965
[13]	validation_0-auc:0.8903	validation_1-auc:0.8869
[14]	validation_0-auc:0.891764	validation_1-auc:0.88836
[15]	va

[141]	validation_0-auc:0.913782	validation_1-auc:0.912
[142]	validation_0-auc:0.913917	validation_1-auc:0.91213
[143]	validation_0-auc:0.91406	validation_1-auc:0.912296
[144]	validation_0-auc:0.914099	validation_1-auc:0.912344
[145]	validation_0-auc:0.914109	validation_1-auc:0.912365
[146]	validation_0-auc:0.914168	validation_1-auc:0.912439
[147]	validation_0-auc:0.914232	validation_1-auc:0.912488
[148]	validation_0-auc:0.914264	validation_1-auc:0.912518
[149]	validation_0-auc:0.91437	validation_1-auc:0.912619
[150]	validation_0-auc:0.914413	validation_1-auc:0.912654
[151]	validation_0-auc:0.914438	validation_1-auc:0.912674
[152]	validation_0-auc:0.914441	validation_1-auc:0.912652
[153]	validation_0-auc:0.914556	validation_1-auc:0.912766
[154]	validation_0-auc:0.914715	validation_1-auc:0.912925
[155]	validation_0-auc:0.9149	validation_1-auc:0.913117
[156]	validation_0-auc:0.914924	validation_1-auc:0.913138
[157]	validation_0-auc:0.914955	validation_1-auc:0.913147
[158]	validation_0-auc

[283]	validation_0-auc:0.925328	validation_1-auc:0.923306
[284]	validation_0-auc:0.925354	validation_1-auc:0.923328
[285]	validation_0-auc:0.925433	validation_1-auc:0.923417
[286]	validation_0-auc:0.925522	validation_1-auc:0.923496
[287]	validation_0-auc:0.925563	validation_1-auc:0.923535
[288]	validation_0-auc:0.925684	validation_1-auc:0.923665
[289]	validation_0-auc:0.925735	validation_1-auc:0.923716
[290]	validation_0-auc:0.925779	validation_1-auc:0.923762
[291]	validation_0-auc:0.925812	validation_1-auc:0.923791
[292]	validation_0-auc:0.925896	validation_1-auc:0.923882
[293]	validation_0-auc:0.925944	validation_1-auc:0.92394
[294]	validation_0-auc:0.926037	validation_1-auc:0.924018
[295]	validation_0-auc:0.926071	validation_1-auc:0.924044
[296]	validation_0-auc:0.926123	validation_1-auc:0.924086
[297]	validation_0-auc:0.926189	validation_1-auc:0.924129
[298]	validation_0-auc:0.926228	validation_1-auc:0.924166
[299]	validation_0-auc:0.926253	validation_1-auc:0.924185
[300]	validatio

[425]	validation_0-auc:0.936012	validation_1-auc:0.934089
[426]	validation_0-auc:0.936077	validation_1-auc:0.934152
[427]	validation_0-auc:0.93615	validation_1-auc:0.934237
[428]	validation_0-auc:0.936207	validation_1-auc:0.934282
[429]	validation_0-auc:0.936305	validation_1-auc:0.934379
[430]	validation_0-auc:0.936391	validation_1-auc:0.934464
[431]	validation_0-auc:0.936455	validation_1-auc:0.934532
[432]	validation_0-auc:0.936633	validation_1-auc:0.934724
[433]	validation_0-auc:0.93665	validation_1-auc:0.934741
[434]	validation_0-auc:0.936728	validation_1-auc:0.934815
[435]	validation_0-auc:0.936765	validation_1-auc:0.934851
[436]	validation_0-auc:0.936811	validation_1-auc:0.934891
[437]	validation_0-auc:0.93695	validation_1-auc:0.935032
[438]	validation_0-auc:0.937006	validation_1-auc:0.935096
[439]	validation_0-auc:0.937038	validation_1-auc:0.935128
[440]	validation_0-auc:0.937099	validation_1-auc:0.935188
[441]	validation_0-auc:0.937108	validation_1-auc:0.935194
[442]	validation_

[567]	validation_0-auc:0.944865	validation_1-auc:0.943259
[568]	validation_0-auc:0.944926	validation_1-auc:0.943315
[569]	validation_0-auc:0.944987	validation_1-auc:0.943377
[570]	validation_0-auc:0.945026	validation_1-auc:0.943418
[571]	validation_0-auc:0.945031	validation_1-auc:0.943424
[572]	validation_0-auc:0.945093	validation_1-auc:0.943492
[573]	validation_0-auc:0.945141	validation_1-auc:0.943537
[574]	validation_0-auc:0.945173	validation_1-auc:0.943567
[575]	validation_0-auc:0.945201	validation_1-auc:0.943599
[576]	validation_0-auc:0.945269	validation_1-auc:0.94366
[577]	validation_0-auc:0.945318	validation_1-auc:0.943708
[578]	validation_0-auc:0.945374	validation_1-auc:0.943764
[579]	validation_0-auc:0.945453	validation_1-auc:0.943844
[580]	validation_0-auc:0.945513	validation_1-auc:0.943911
[581]	validation_0-auc:0.945559	validation_1-auc:0.943953
[582]	validation_0-auc:0.945611	validation_1-auc:0.944004
[583]	validation_0-auc:0.94567	validation_1-auc:0.944063
[584]	validation

...

[3514]	validation_0-auc:0.992655	validation_1-auc:0.992295
[3515]	validation_0-auc:0.992657	validation_1-auc:0.992297
[3516]	validation_0-auc:0.992663	validation_1-auc:0.992302
[3517]	validation_0-auc:0.992664	validation_1-auc:0.992304
[3518]	validation_0-auc:0.992666	validation_1-auc:0.992306
[3519]	validation_0-auc:0.992669	validation_1-auc:0.992309
[3520]	validation_0-auc:0.992671	validation_1-auc:0.992311
[3521]	validation_0-auc:0.992677	validation_1-auc:0.992316
[3522]	validation_0-auc:0.992681	validation_1-auc:0.992321
[3523]	validation_0-auc:0.992684	validation_1-auc:0.992324
[3524]	validation_0-auc:0.992689	validation_1-auc:0.992329
[3525]	validation_0-auc:0.992692	validation_1-auc:0.992332
[3526]	validation_0-auc:0.992695	validation_1-auc:0.992335
[3527]	validation_0-auc:0.992701	validation_1-auc:0.992341
[3528]	validation_0-auc:0.992706	validation_1-auc:0.992347
[3529]	validation_0-auc:0.992709	validation_1-auc:0.99235
[3530]	validation_0-auc:0.992713	validation_1-auc:0.99235

[3654]	validation_0-auc:0.993154	validation_1-auc:0.992816
[3655]	validation_0-auc:0.993159	validation_1-auc:0.992822
[3656]	validation_0-auc:0.993161	validation_1-auc:0.992823
[3657]	validation_0-auc:0.993171	validation_1-auc:0.992834
[3658]	validation_0-auc:0.993178	validation_1-auc:0.992842
[3659]	validation_0-auc:0.993181	validation_1-auc:0.992845
[3660]	validation_0-auc:0.993189	validation_1-auc:0.992854
[3661]	validation_0-auc:0.993194	validation_1-auc:0.992857
[3662]	validation_0-auc:0.993197	validation_1-auc:0.992861
[3663]	validation_0-auc:0.993198	validation_1-auc:0.992861
[3664]	validation_0-auc:0.993203	validation_1-auc:0.992866
[3665]	validation_0-auc:0.993207	validation_1-auc:0.99287
[3666]	validation_0-auc:0.993208	validation_1-auc:0.992872
[3667]	validation_0-auc:0.993211	validation_1-auc:0.992875
[3668]	validation_0-auc:0.993214	validation_1-auc:0.992878
[3669]	validation_0-auc:0.993216	validation_1-auc:0.992882
[3670]	validation_0-auc:0.993217	validation_1-auc:0.99288

[3794]	validation_0-auc:0.993595	validation_1-auc:0.993279
[3795]	validation_0-auc:0.993597	validation_1-auc:0.993281
[3796]	validation_0-auc:0.993602	validation_1-auc:0.993285
[3797]	validation_0-auc:0.993605	validation_1-auc:0.993289
[3798]	validation_0-auc:0.993607	validation_1-auc:0.993291
[3799]	validation_0-auc:0.993607	validation_1-auc:0.993292
[3800]	validation_0-auc:0.993611	validation_1-auc:0.993295
[3801]	validation_0-auc:0.993613	validation_1-auc:0.993297
[3802]	validation_0-auc:0.993616	validation_1-auc:0.993301
[3803]	validation_0-auc:0.993617	validation_1-auc:0.993301
[3804]	validation_0-auc:0.993622	validation_1-auc:0.993307
[3805]	validation_0-auc:0.993624	validation_1-auc:0.993309
[3806]	validation_0-auc:0.993627	validation_1-auc:0.993313
[3807]	validation_0-auc:0.993631	validation_1-auc:0.993317
[3808]	validation_0-auc:0.993632	validation_1-auc:0.993319
[3809]	validation_0-auc:0.993634	validation_1-auc:0.99332
[3810]	validation_0-auc:0.993638	validation_1-auc:0.99332

[3934]	validation_0-auc:0.993992	validation_1-auc:0.99369
[3935]	validation_0-auc:0.993994	validation_1-auc:0.993693
[3936]	validation_0-auc:0.993997	validation_1-auc:0.993695
[3937]	validation_0-auc:0.994003	validation_1-auc:0.993701
[3938]	validation_0-auc:0.994002	validation_1-auc:0.993701
[3939]	validation_0-auc:0.994003	validation_1-auc:0.993702
[3940]	validation_0-auc:0.994009	validation_1-auc:0.993708
[3941]	validation_0-auc:0.994015	validation_1-auc:0.993714
[3942]	validation_0-auc:0.994021	validation_1-auc:0.99372
[3943]	validation_0-auc:0.994023	validation_1-auc:0.993722
[3944]	validation_0-auc:0.994027	validation_1-auc:0.993726
[3945]	validation_0-auc:0.99403	validation_1-auc:0.99373
[3946]	validation_0-auc:0.994035	validation_1-auc:0.993735
[3947]	validation_0-auc:0.994035	validation_1-auc:0.993735
[3948]	validation_0-auc:0.994037	validation_1-auc:0.993738
[3949]	validation_0-auc:0.994038	validation_1-auc:0.993739
[3950]	validation_0-auc:0.994038	validation_1-auc:0.99374
[3

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=4000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.03, scale_pos_weight=1, seed=123,
              silent=None, subsample=0.9, verbosity=1)

In [67]:
preds7 = xg_cl7.predict(test_enc)

In [68]:
sample7 = pd.read_csv('sample_submission2.csv')

In [69]:
sample7.EmployeeNo = test.EmployeeNo

In [70]:
sample7.Promoted_or_Not = preds7

In [71]:
sample7.to_csv('intercampus7.csv', index = False)

In [72]:
value_count(features, features.columns)


For column                                     EmployeeNo
38312

For column                                     Division
9

For column                                     Qualification
3

For column                                     Gender
2

For column                                     Channel_of_Recruitment
3

For column                                     Trainings_Attended
10

For column                                     Last_performance_score
6

For column                                     Targets_met
2

For column                                     Previous_Award
2

For column                                     Training_score_average
61

For column                                     State_Of_Origin
3

For column                                     Foreign_schooled
2

For column                                     Marital_Status
3

For column                                     Past_Disciplinary_Action
2

For column                                     Previous_IntraDep

In [73]:
features['Year_of_recruitment_Str'].value_counts()

2011-2015    18092
2016-2018    12766
2006-2010     5023
2001-2005     1754
1982-2000      677
Name: Year_of_recruitment_Str, dtype: int64